<a href="https://colab.research.google.com/github/a1ire3a/SVM-from_Scratch/blob/main/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import random as rnd
import cvxopt
from cvxopt import matrix as cvxopt_matrix
from numpy import linalg
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from scipy.stats import mode
from numpy.ma.core import append
from sklearn.datasets import load_breast_cancer

In [ ]:
cvxopt.solvers.options["show_progress"] = False

In [ ]:
def linear_kernel(x,z,args):
  return np.dot(x.T,z)

def polynomial_kernel(x,z,args):
  degree = args['degree']
  result = (1.0 + np.dot(x.T,z)) ** degree
  return result

def rbf_kernel(x,z,args):
    gamma = args['gamma']
    return np.exp(-1 * gamma * (linalg.norm(x-z)**2))

In [ ]:
class SVM(object):

    def __init__(self, kernel=linear_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Gram matrix
        y = y.reshape(-1,1) * 1.
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j], hyper_parameters)
        #Converting into cvxopt format
        #print(K)
        m,n = X.shape
        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt_matrix(-np.ones((m, 1)))
        G = cvxopt_matrix(np.vstack((np.eye(m)*-1,np.eye(m))))
        h = cvxopt_matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        A = cvxopt_matrix(y.reshape(1, -1))
        b = cvxopt_matrix(np.zeros(1))
        # solve QP problem
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        
        # Lagrange multipliers
        alpha = np.ravel(solution['x'])
        
        # Support vectors have non zero lagrange multipliers
        sv = alpha > 1e-5

        #this will actually give the indices of the support vectors
        ind = np.arange(len(alpha))[sv]
        
        # get alphas of support vector , Xs and ys too.
        self.alpha = alpha[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        alpha = self.alpha
        alpha = alpha.reshape(-1,1)
        # Intercept
        self.b = 0
        for n in range(len(alpha)):
            self.b += self.sv_y[n]
            self.b -= np.sum(alpha * self.sv_y * K[ind[n],sv])
            self.b /= len(self.alpha)
        
        # Weight vector
        if self.kernel == linear_kernel:
            self.w = ((y[sv] * alpha).T @ X[sv,:]).reshape(-1,1)
        else:
            self.w = None #Guidance: for non-linear case this should be None. 

    def predict(self, X):
        if self.w is not None:
            return np.sign(np.dot(X, self.w) + self.b)
        else:
            s = 0
            for a, sv_y, sv in zip(self.alpha, self.sv_y, self.sv):
                s += a * sv_y * self.kernel(X, sv, hyper_parameters)
                        
            return np.sign(s + self.b)

    def args(self):
        return (self.w, self.b)


In [ ]:
class multiClassSVM(object):

    def __init__(self, kernel=linear_kernel, C=None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)

    def fit(self,X,y): #One VS One
        self.model = []
        for i in np.unique(y):
              for j in np.unique(y):
                  y_oVo = []
                  X_oVo = []
                  if j <= i:
                      continue
                  else:
                      for k in range(len(y)):
                            if y[k] == i:
                                y_oVo.append(1)
                                X_oVo.append(X[k])
                            elif y[k] == j:
                                y_oVo.append(-1)
                                X_oVo.append(X[k])

                      y_oVo = np.asarray(y_oVo)
                      X_oVo = np.asarray(X_oVo)

                      temp = SVM(self.kernel, self.C)
                      temp.fit(X_oVo, y_oVo)
                      self.model.append((temp, i, j))
    
    def predict(self,X):
        X_predicted = []
        for i in X:
            pred = []
            for temp, c1, c2 in self.model:
                if temp.predict(i) == +1:
                    pred.append(c1)
                elif temp.predict(i) == -1:
                    pred.append(c2)
                
            X_predicted.append(mode(pred)[0])
            
        return np.asarray(X_predicted)

    def args(self):
        for temp, c1, c2 in self.model:
            return (temp.args(i), c1, c2)
     
    '''def fit(self, X, y): #One VS All
        self.model = []
        for i in np.unique(y):
            y_oVa = np.where(y == i,1, -1)
            temp = SVM(self.kernel, self.C)
            temp.fit(X, y_oVa)
            self.model.append((temp, i))  

    def predict(self, X):
        pred = []
        for i in X:
            for temp, c in self.model:
                if temp.predict(i) == 1:
                    pred.append(c)
                    break
        return pred'''

# ***1***

In [ ]:
dataset = fetch_olivetti_faces()
faces = dataset.images
target = dataset.target
X = []
for i in faces:
    X.append(np.ndarray.flatten(i))
X = np.asarray(X)
face_train, face_test, target_train, target_test = train_test_split(X, target, test_size=0.2)

C = 0.001
hyper_parameters = {}
model = multiClassSVM(kernel=linear_kernel,C=C)
model.fit(face_train, target_train)
y_pred = model.predict(face_test)
acc = accuracy_score(target_test, y_pred)
print(acc)

0.05


In [ ]:
iris = load_iris()
x = iris.data
yy = iris.target
std = StandardScaler()
xx = std.fit_transform(x)
iris_train, iris_test, it_train, it_test = train_test_split(xx, yy, test_size=0.2)

C = 0.005
hyper_parameters = {}
model = multiClassSVM(kernel=linear_kernel,C=C)
model.fit(iris_train, it_train)
y_pred = model.predict(iris_test)
acc = accuracy_score(it_test, y_pred)
print(acc)

0.7


In [ ]:
bs = load_breast_cancer()
data = bs.data
target = bs.target
data_s = StandardScaler().fit_transform(data)
bd_train, bd_test, bt_train, bt_test = train_test_split(data_s, target, test_size=0.2)

C = 1
hyper_parameters = {}
model = multiClassSVM(kernel=linear_kernel,C=C)
model.fit(bd_train, bt_train)
y_pred = model.predict(bd_test)
acc = accuracy_score(bt_test, y_pred)
print(acc)

0.9736842105263158


# ***2***

In [ ]:
y = iris.target
for i in range(4):
    for j in range(4):
        if i < j:
            print('Select feature', i+1, 'and', j+1, 'from iris dataset:')
            X = StandardScaler().fit_transform(iris.data[:, (i,j)])
            iris_train, iris_test, it_train, it_test = train_test_split(X, y, test_size=0.2)
            C = 1
            hyper_parameters = {'degree':5}
            model = multiClassSVM(kernel=polynomial_kernel,C=C)
            model.fit(iris_train,it_train)
            y_pred = model.predict(iris_test)
            acc = accuracy_score(it_test, y_pred)
            print('Accuracy:', acc)

Select feature 1 and 2 from iris dataset:
Accuracy: 0.6333333333333333
Select feature 1 and 3 from iris dataset:
Accuracy: 0.8
Select feature 1 and 4 from iris dataset:
Accuracy: 0.6333333333333333
Select feature 2 and 3 from iris dataset:
Accuracy: 0.6
Select feature 2 and 4 from iris dataset:
Accuracy: 0.8333333333333334
Select feature 3 and 4 from iris dataset:
Accuracy: 0.7


In [ ]:
y = iris.target
for i in range(4):
    for j in range(4):
        if i < j:
            print('Select feature', i+1, 'and', j+1, 'from iris dataset:')
            X = StandardScaler().fit_transform(iris.data[:, (i,j)])
            iris_train, iris_test, it_train, it_test = train_test_split(X, y, test_size=0.2)
            C = 36
            hyper_parameters = {'gamma':9}
            model = multiClassSVM(kernel=rbf_kernel,C=C)
            model.fit(iris_train,it_train)
            y_pred = model.predict(iris_test)
            acc = accuracy_score(it_test, y_pred)
            print('Accuracy:', acc)

Select feature 1 and 2 from iris dataset:
Accuracy: 0.5
Select feature 1 and 3 from iris dataset:
Accuracy: 0.9666666666666667
Select feature 1 and 4 from iris dataset:
Accuracy: 0.9
Select feature 2 and 3 from iris dataset:
Accuracy: 0.9
Select feature 2 and 4 from iris dataset:
Accuracy: 0.8666666666666667
Select feature 3 and 4 from iris dataset:
Accuracy: 0.9666666666666667
